In [18]:
import numpy as np

def forward_algorithm(observations, hidden_states, initial_probs, transition_probs, emission_probs):
    # Number of hidden states
    num_states = len(hidden_states)
    # Length of the observation sequence
    sequence_length = len(observations)

    # Initialize the forward probabilities alpha
    alpha = np.zeros((sequence_length, num_states))
    for i in range(num_states):
        alpha[0][i] = initial_probs[i] * emission_probs[i][observations[0]]

    # Recursively compute the forward probabilities for each time step
    for t in range(1, sequence_length):
        for j in range(num_states):
            alpha[t][j] = emission_probs[j][observations[t]] * sum(alpha[t - 1][i] * transition_probs[i][j] for i in range(num_states))

    # Compute the total probability of observing the sequence
    probability = sum(alpha[sequence_length - 1])

    return alpha, probability

In [19]:
obs = [1, 2, 1, 2]
hidden = ['A', 'B']  # possible hidden states
initial_probs = np.array([0.35, 0.75])  # initial state probabilities
transition_probs = np.array([[0.65, 0.45], [0.25, 0.85]])  # transition probabilities
emission_probs = np.array([[0.15, 0.95, 0.45], [0.75, 0.35, 0.65]])  # emission probabilities

# Compute the forward probabilities and the total probability of observing the given sequence
alpha, probability = forward_algorithm(obs, hidden, initial_probs, transition_probs, emission_probs)
print("Forward probabilities:\n", alpha)
print("Total probability of observing sequence {} in the given HMM: {:.4f}".format(obs, probability))

Forward probabilities:
 [[0.3325     0.2625    ]
 [0.1267875  0.2422875 ]
 [0.13583456 0.09204956]
 [0.05008719 0.09058899]]
Total probability of observing sequence [1, 2, 1, 2] in the given HMM: 0.1407


In [20]:
import numpy as np

def viterbi(observations, hidden_states, initial_probabilities, transition_probabilities, emission_probabilities):
    num_states = len(hidden_states)
    sequence_length = len(observations)

    # Initialize the Viterbi probabilities delta and the corresponding backpointers
    delta = np.zeros((sequence_length, num_states))
    backpointers = np.zeros((sequence_length, num_states), dtype=int)
    for i in range(num_states):
        delta[0][i] = initial_probabilities[i] * emission_probabilities[i][observations[0]]

    # Recursively compute the Viterbi probabilities and backpointers for each time step
    for t in range(1, sequence_length):
        for j in range(num_states):
            max_delta = 0
            max_backpointer = 0
            for i in range(num_states):
                current_delta = delta[t - 1][i] * transition_probabilities[i][j] * emission_probabilities[j][observations[t]]
                if current_delta > max_delta:
                    max_delta = current_delta
                    max_backpointer = i
            delta[t][j] = max_delta
            backpointers[t][j] = max_backpointer

    # Find the most likely sequence of hidden states
    most_likely_sequence = [0] * sequence_length
    most_likely_sequence[-1] = np.argmax(delta[sequence_length - 1])
    for t in range(sequence_length - 2, -1, -1):
        most_likely_sequence[t] = backpointers[t + 1][most_likely_sequence[t + 1]]

    return most_likely_sequence

In [21]:
obs = [1, 2, 1, 0]
hidden = ['A', 'B']  # possible hidden states
initial_probs = np.array([0.4, 0.6])  # initial state probabilities
transition_probs = np.array([[0.7, 0.3], [0.2, 0.8]])  # transition probabilities
emission_probs = np.array([[0.15, 0.85, 0.45], [0.75, 0.35, 0.65]])  # emission probabilities

# Compute the most likely sequence of hidden states for the given sequence of observations
most_likely_sequence = viterbi(obs, hidden, initial_probs, transition_probs, emission_probs)
print("Most likely sequence of hidden states for observed sequence {}: {}".format(obs, [hidden[i] for i in most_likely_sequence]))

Most likely sequence of hidden states for observed sequence [1, 2, 1, 0]: ['B', 'B', 'B', 'B']
